In [46]:
import pandas as pd 
import os
import json

In [38]:
df = pd.read_csv("./data/Kickstarter_merged.csv")

In [39]:
# convert 'video' to a binary categorical variable
df['video'].value_counts()
df['has_video'] = df['video'].apply(lambda x: 0 if pd.isnull(x) else 1)
df['has_video'].value_counts()

1    1404
0     728
Name: has_video, dtype: int64

In [44]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_5579/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [47]:
with open("data/0_18814_story.json", 'r') as f:
    story = json.load(f)
with open("data/0_18814_risk.json", 'r') as f1:
    risk = json.load(f1)

In [48]:
print(len(story), len(risk))

18815 18815


In [53]:
df_subset = df_1.iloc[:18815]
print(len(df_subset))

df_subset['story'] = story
df_subset['risk'] = risk

18815


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_5579/632398381.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['story'] = story
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_5579/632398381.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['risk'] = risk


In [60]:
df_subset.iloc[0]

backers_count                                                              33
blurb                       Ademi bar has succeeded in selling its first 5...
category                    {"id":315,"name":"Vegan","analytics_name":"Veg...
converted_pledged_amount                                               4095.0
country                                                                    US
country_displayable_name                                    the United States
created_at                                                         1591815974
creator                     {"id":1825832733,"name":"Aidar and Aizhan","sl...
currency                                                                  USD
currency_symbol                                                             $
currency_trailing_code                                                   True
current_currency                                                          USD
deadline                                                        